___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios-superiores-de-occidente.png" width="150px" height="100px" />

# Microestructuras y Sistema de Trading
## Laboratorio 1

### EQUIPO:
#### - Tere
#### - Ana Laura Correa
#### - Gaspar Benitez
#### - Marysol Cantarero

##### Situación: 
Acabas de entrar a trabajar a una operadora de fondos de inversión como analista financiero dentro de la mesa de análisis del mercado de renta variable. En la junta trimestral se plantea la posibilidad de crear un fondo propio con exposición al mercado de valores de USA tratando de replicar a los ETFs conocidos como ishares que la empresa BlackRock ofrece. 

La idea es replicar un ETF con posiciones en directo. Te han asignado la tarea de hacer un ejercicio simple de replicar la construcción de un portafolio de inversión y un criterio de rebalanceo lo más parecido a un ETF de ishares que tu selecciones. Esto con la finalidad de comparar los resultados de haber rebalanceado periódicamente el portafolio, ó, no haberlo hecho y continuar todo un año con las mismas posiciones.


##### Características mínimas de solución: 
Debido a tu formación, te piden que este proceso que construyas sea repetible para cualquier ETF y también incluir 3 medidas de atribución al desempeño, tanto para el caso que el ETF nunca hubiese sido rebalanceado como para los casos donde sí. 

En la mesa de análisis están interesados en conocer el proceso que sigues para adquirir, limpiar, dar formato y utilizar los datos, y a pesar que el equipo senior de análisis está compuesto por personal altamente calificado en temas de economía y finanzas, ellos no saben programar por lo que te piden que incluyas el código utilizado, y que sea de manera ordenada y explicando el sentido de las funciones y paqueterías que requeriste. 

Comparar con 1 año de información: 1.8.18 al 1.8.19 

Utilizar un ETF de Ishares para el mercado de renta varialbe de USA 

Entregar un Jupyter Notebook con explicación y comentarios en el código 

Crear y actualizar continuamente un repositorio en GitHub para el proyecto.

### RESOLUCIÓN DE PROBLEMA

Los ETF(Exchange Trade Funds) son fondos o instrumentos de inversión cuyos títulos cotizan en la bolsa, al igual que una acción bursátil,pudiendose comprar y vender en cualquier momento, sin necesidad de esperar al cierre del mercado. Su objetivo es replicar el comportamiento de los activos que componen un índice bursátil, ya sea de renta fija, variable, divisas o materias primas u otros activos financieros.

Para este problema se analizara el ETF "iShares U.S. Financials ETF (IYF)" para el mercado de renta variable de USA con posiciones en directo tomando información del 01/Agosto/2018 al 01/Agosto/2019. Sin embargo, el código a realizar debe ser repetible para cualquier ETF.

Las 3 medidas de atribución al desempeño que se eligieron para la comparación de los resultados son: Rendimiento, volatilidad y medidas de desempeño.

In [4]:
# DESCARGA DE LIBRERIAS
import quandl
import datetime
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

In [5]:
# FUNCIÓN PARA DESCARGA DE ARCHIVOS CON DATOS ETF (ISHARES)
def holdings_csv(narch,name):
    data = []
    for i in range(narch):
        r = name+str(i)+".csv"
        info = pd.read_csv(r,header=10, index_col= 0)
        info = info.dropna() # quitando los datos vacios.
        info = info.drop(columns = ['Name', 'Asset Class', "Market Value", 'Notional Value', 'Sector', 'SEDOL', 'ISIN', 'Exchange'])
        data2 = data.append(info)
    return data
holdings = holdings_csv(13,"IYF_holdings")

In [9]:
# FUNCIÓN PARA DESCARGA DE DATOS HISTÓRICOS (QUANDL)
quandl.ApiConfig.api_key = "My9ygJsz7yTDx_9jcneH"  # Quandl API key
fechas = list(['2017-02-28', '2017-03-31', '2017-04-28', '2017-05-31', '2017-06-30',
                 '2017-07-31', '2017-08-31', '2017-09-29', '2017-10-31', '2017-11-30',
                 '2017-12-29', '2018-01-31', '2018-02-28'])
num_files = 13

In [7]:
index= holdings[0].index
q, h = holdings[0].shape


In [10]:
# Functions for the Model
def download_quandl(fechas, num_files):
    """
    Function to Download data from Quandl
    :param dates: list of Dates to download
    :param num_files: number of archives
    :return: DF dictionary of Data
    """
    d2 = {}  # Empty dictionary
    for i in range(num_files):  # Loop to Download data from Quandl
        index= holdings[i].index
        q, h = holdings[i].shape
        ticker_r = list(index)  # ticker = 'WIKI/AAPL'
        date = {'gte': fechas[i], 'lte': fechas[i] }
        column = {'columns': ['ticker', 'date', 'Adj_close']}
        data = quandl.get_table('WIKI/PRICES', qopts=column, ticker=ticker_r, date=date, paginate=True)
        d2["Data_{0}".format(i+1)] = data.set_index('ticker')
        d[nom] = d[nom].set_index('Ticker')  # Ticker is set as index
    return d2

In [12]:
download_quandl(fechas,num_files)

NameError: name 'd' is not defined

In [17]:
quandl.ApiConfig.api_key = "ekAPZHdA3_kmwr25TxsE"
index= holdings[0].index
q, h = holdings[0].shape
ticker_r = list(index)  # ticker = 'WIKI/AAPL'
date = {'gte': '2017-08-01', 'lte': '2019-08-01' }
column = {'columns': ['ticker', 'date', 'Adj_close']}
data = quandl.get_table('WIKI/PRICES',  date= date, ticker=ticker_r, paginate=True)
data

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
None,,,,,,,,,,,,,,
0,ZION,2018-03-27,53.25,53.7100,51.6200,52.02,2214940.0,0.00,1.0,53.250000,53.710000,51.620000,52.020000,2214940.0
1,ZION,2018-03-26,52.20,53.4100,51.9000,53.24,2357767.0,0.00,1.0,52.200000,53.410000,51.900000,53.240000,2357767.0
2,ZION,2018-03-23,53.22,53.5500,51.1700,51.23,3378671.0,0.00,1.0,53.220000,53.550000,51.170000,51.230000,3378671.0
3,ZION,2018-03-22,54.53,54.7000,52.9700,53.07,2578239.0,0.00,1.0,54.530000,54.700000,52.970000,53.070000,2578239.0
4,ZION,2018-03-21,54.94,55.7800,54.3900,55.10,1992966.0,0.00,1.0,54.940000,55.780000,54.390000,55.100000,1992966.0
5,ZION,2018-03-20,55.35,55.4712,54.4700,54.80,2643688.0,0.00,1.0,55.350000,55.471200,54.470000,54.800000,2643688.0
6,ZION,2018-03-19,55.32,55.7699,54.3400,54.99,1634460.0,0.00,1.0,55.320000,55.769900,54.340000,54.990000,1634460.0
7,ZION,2018-03-16,55.00,55.8000,54.9200,55.52,5424358.0,0.00,1.0,55.000000,55.800000,54.920000,55.520000,5424358.0
8,ZION,2018-03-15,55.57,55.5700,54.4700,54.75,2116564.0,0.00,1.0,55.570000,55.570000,54.470000,54.750000,2116564.0


### CONSTRUCCIÓN DE PORTAFOLIO DE INVERSIÓN

In [64]:
w = holdings[0]['Weight (%)']# pesos
p = list(holdings[0]['Price'])   # precios
s = list(holdings[0]['Shares'])  #número de acciones
#a = p*w*s # Valor en dinero de cada acción
#v_etf_total = sum(a) #Valor del portafolio o etf, suma de los valores de las acciones

In [ ]:
# Desviación estandar
nav = pd.read_excel('iShares-US-Financials-ETF_fund.xls',sheet_name=1, index_col= 0)
nav = nav["NAV per Share"][11:263]
#nav = list(nav)
#nav.reverse()
desv1 = np.std(nav)
mean1 = np.mean(nav)

### CRITERIO DE RE BALANCEO.

In [ ]:
#SERIE DE TIEMPO HACIENDO RE BALANCEO
plt.title("Serie de tiempo ETF IYF")
plt.ylabel("NAV")
plt.xlabel("Fechas")
plt.axvline(x=0, c = "green")
plt.axvline(x=22, c = "greenyellow")
plt.axvline(x=43, c = "tomato")
plt.axvline(x=64, c = "lightcoral")
plt.axvline(x=85, c = "aqua")
plt.axvline(x=106, c = "plum")
plt.axvline(x=125, c = "mediumvioletred")
plt.axvline(x=147, c = "lawngreen")
plt.axvline(x=166, c = "greenyellow")
plt.axvline(x=186, c = "gold")
plt.axvline(x=209, c = "teal")
plt.axvline(x=229, c = "lavender")
plt.axvline(x=251, c = "darkmagenta")
plt.plot(nav, c = "k")

### COMPARACIÓN DE RESULTADOS: REBALANCEO VS. MISMAS POSICIONES

In [ ]:
# MEDIDAS DE ATRIBUCIÓN AL DESEMPEÑO

### CONCLUSIONES
.....

### BIBLIOGRAFÍA
https://www.portafolio.co/economia/finanzas/son-etf-293754

https://economipedia.com/definiciones/etf-fondos-cotizados.html

https://www.ishares.com/us/products/239508/#/

https://www.quandl.com/tools/python

